# Playing with RSA

In [ ]:
from math import log2, floor, isqrt
from timeit import timeit
from sympy import randprime, nextprime

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization

## What happens if p and q are too close?

In [ ]:
# p = next_prime(2^512)
p = randprime(2**511, 2**512)
p

In [ ]:
q=nextprime(p)
q

In [ ]:
n = p * q
n

In [ ]:
# n has 1024 bit
log2(n)

In [ ]:
# We start from the square root and look for the next prime
# isqrt is the integer part of the square root
t = isqrt(n)
t

In [ ]:
nextprime(t)

In [ ]:
# Compare with q
q

## How long does it take to generate a key?

In [ ]:
def gen():
    return rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )

In [ ]:
timeit(gen, number = 100)

## What happens if the exponent is too low?

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=3,
    key_size=1024
)
public_key = private_key.public_key()

e = public_key.public_numbers().e
n = public_key.public_numbers().n
print("e = {}\nn = {}".format(e,n))

In [ ]:
message = b"OK"

m = int.from_bytes(message,"big")
c_small = pow(m,e,n)

print("m = {}\nc = {}".format(m,c_small))

In [ ]:
# Can we find the message?
# If the message is small, the modulo is not effective and we can just take the e-th root
pow(c_small,1/3)

In [ ]:
# If we use padding the problem is solved
c = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1( algorithm=hashes.SHA256() ),
        algorithm=hashes.SHA256(),
        label=None
    )
)
print("c = {:d}".format(int.from_bytes(c,"big")))

In [ ]:
private_key.decrypt(
    c,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

## Signature

The key generation is the same as encryption, but we want a fresh key

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096
)
public_key = private_key.public_key()

In [ ]:
message = b"A message I want to sign"
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1( hashes.SHA256() ),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

In [ ]:
try:
    public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1( hashes.SHA256() ),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("OK")
except:
    print("KO")

In [ ]:
message = b"Not the real message"
try:
    public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1( hashes.SHA256() ),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("OK")
except:
    print("KO")

## How to export and import the public key

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096
)
public_key = private_key.public_key()

In [ ]:
#Export public key
pem = public_key.public_bytes(
   encoding=serialization.Encoding.PEM,
   format=serialization.PublicFormat.SubjectPublicKeyInfo
)
pem

In [ ]:
#Import public key
pem = b'-----BEGIN PUBLIC KEY-----\nMIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEAu/Av7jMZ66oaSWZXHs+J\nWlWcL2c4TjWDVS0qVCAquQRNY/DPNO7SWP6rIdPybKtOdPLf0L6mjHZ374UzxuU4\nN4dCanKeVAz7spf6JqTJCxL9mao7cD83hRIhXYNp1ljNJKw8E9U/X8m3uFOnEiZb\nDswPEudBQqu98zKUDOlIZl2/UPk6CO5ocs384A5sXVHUB7a1rb3Fw9x9HEZosm7o\nx/hzskU11OvVux2nQxIeiYdIimRGKmHiSMbWY5LI5QEn8aPcUwK2e+ZUa8o9khQr\n5EpzBZBUzxk7UC3iWDG+pmGkI0AHdExir8nRuhQHIm/TxTl2aLwZOnDjs6gOzEn5\npsHhEsx8wYktnQnZV4ix59SPbhxhW9mo3b4F+04k7/JPpF/WaWhK+GGTRtZFs19A\nrF5p+EIoCr9PEaL5nhDTKZm9Zl73Bor/VyZxi6DpBU/NVdWV6B1kgEoU/yvR0JM6\nMisBQQwxzJtNs/P0qkSbK+KzS37ANadETktVBRYUl213ihY/z0RrYoA/dguHWYYV\nbONHaBAb0HDIx0dk26RnjXetY06bYHF8WMhWE9YmvwUoz+9Fy6Yis6Xw+2u37tR3\nHxyVcRdKq4kavYaoBRTGyFf5iz1AOPVTJzHOGkLt+oisBwQoH09DiZL4sZ2YA4lE\nfcD0yRbK6We+xdN+aYI35jUCAwEAAQ==\n-----END PUBLIC KEY-----\n'
rsa_public_key = serialization.load_pem_public_key(pem)

# Laboratory

1. Measure the key generation time and the signature time for key sizes ranging from 512 to 4096 bits. Plot a graph.

2. The following is my public key. Verify my message "Hallo class!"

In [ ]:
# My public key
b'-----BEGIN PUBLIC KEY-----\nMIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEA40GwUqIGKVgJ3aDjyeuh\n+IjgAYS7ZIUDmWF6zCA6/1ybbjkp3Hj6n7MYyN2VajK88Qc1EQVKiDxpnbMpc2bE\nhrC+ElyEoI+Vy5R6CVQ8UtKEFwQ+dFgcMOrsw/4v1lj0RyQpo2yzwXY8aXqpraKG\nyyKEu+gbmUpjKJODAyyPey8xM3LyWlzB21QwVYO+6H9kQ48engFJMBT2di5gmlZv\ngNOKfdA+EWi3biblQlOPUSTtvMNzIlH1XIO27t21yQK3yTLB6+uZjjFe5s5h7uTU\nRdbrOksTD+4DGzFAa8o7Uom34BRZJ0zUIcZVBqwvX/8NA7pSRoL9Q8kq0hCcnZak\n2fgRoAK26nBzO1BcsvrnfHwr2u/lg/5Sj86d9mFRCIfAzEXPm+1Gf3oBzfxZzgEs\nV4O1IjRmI8pul0M77rWoC7NCodZk5VbIniEY3FOKajsB1WVlZF47EuwqwkDekZBN\nSJpPAWuModDBwTakBCznNsFFrgIS36DVtEYIgCdQA7n444t7qu3CmdktCgD+MruB\n5Qe+8Qcn6aLlEyYwykj8vfaI06ztdCDuIdB7aeWAqb22luXxHgNGuIIGHPsOuUHF\nJqLJjFBTf8gQKhv6ryIEeACYvDjWuN9xbvzRhM8IFIE5yw6xUuMbrhSfJYbNATU8\naq2/Ut7ZungeWwW3TILiW1ECAwEAAQ==\n-----END PUBLIC KEY-----\n'

In [ ]:
# The signature of my message "Hallo class!"
b'}\xf3\x0b\xa3\xda\xd4a\xe0@p\xdfz\xb0\xdbY\xc9\x10:\xbe\xae\x84!\xd5e\xf4\xe2\xebD\xb4\xd4H\xcd\x1b\xf4r\x7f\xec\x0e\x83\xa7\x9a\xdbY\x8d\xdd\xc9\xect\xea\x1b\x01\x02\xbd\xb5\x9dYz\xd9\x0fv\xf7\x8d\xbc\xa1\x1d\x80\x91}\xd6\xbc:\xbb\xce\x00o\xa8\'\x08@\x99\xfa\xec;a\xb2lEy]\x9a\xccV\x00\xb0\x8d\x9a\xf0F\x0f\x81\x8f\xf7\xb8\xf8\xc7i,\x0bCn93\xf2\x7f\x08\xbb\x00\xe9]\xb8\xcc\x04i\x93\xd6y\xbc\xa9N\xe7\xb5tI\xe2\x9a\xc4{4\x8c\xa6&\xd4b\xa6\xdf\x10I\xe7ZI3\xee)\xba_\x0c)\xa3\x04\xd7\xe1\'\xcaA]J\x1d\xf0?M\xf2\xb1Ft\x9a\xad\xf6y\x8a3\x87\xc02\xbe%\xfaOk\xfa\x99\xe6\x14\x82\x07wN\xe9%\xd5\xbd\xd4\x84\x0b]\xd3\xfa\xab#\xf9\x84\xf9U\xbd\x0f\xa0\x85NL!\x9f\xda\xf18\xad\x81\x0fg(\x94\xd8mt\x98Z\xed`t\tm\xf1O\x85\x1f\xa8\x7f\xb0\x1d\x87\xf9c\x89\x1f\xb6$\x05\xd3Z\xab\xc0a\x85\xe9\x8e\x91\xbb\xf9\xe8\x99\xec(\xc0\xd7\xdcf\xbf|\x8e.\xf8\xd7H\x04\xec\n\xffDKq\x94\xb9\xe4\r\xd6r\xa1Gf=V\x16%\x1e\x9e\xbc\xb3\xabQ \xd0\x19\x92~\x19?]\x15\xf3"\xdc\x9876\x8f\x8a\xca\\\x153d\xb6\xab:\xc8\xec)Q\xcb*\xac\x92\xc3\x06\xe9A\x92\x95\x1a\xae\xcf\xf5\x95<6\x94n\xf9\xd8_\xbf\x05\xc4T\xc3\xed\xaf\xf9IK\x8e\x16B\x95\x15\xdd\x8a\xb0V\xa8^\xfd\x06:\x08)\xcff\xe9uY\xee\xad)\x07S}\x9b\xb0\xf4\xd9\xa7\x92m\x84\xe5\xab\x9c\x84\x96k\xc2+Y\x0f\x9ci#p\xd6z\xd6\x81\xe9\xb2D\x99h\'z\x0b\xd0\xe4W\x8a|C\xea\xf3\x94\xb8}\x87t\x13\xf5u\xbd\xcb\x89\t\xb6\xa1a\x8a\x9cC\r\xab\xd3\xa1\xdf\xffW\xe0LY\x18\x83\x05*\x85\xa9u\xf4\xb4\xe4\xfc\xda\xb4JB~\xdeTd\x07\xcc\xed\x1a\x80\x08`\xb1\xefB\xf9j:\x85\xbc\xb0\x94\x8dbR\xe8\n>w8\xf6\x11F'

3. Work in pairs. One plays as Alice, one as Bob.
Alice generates a keypair and sends her public key to Bob. Bob encrypts a message and sends it to Alice.
Alice decrypts the message.

4. Work in pairs. One plays as Alice, one as Bob. Bob generates a keypair and sends his public key to Alice.
Alice generates a random key and sends it to Bob using RSA. Then Alice encrypts a message using AES-CTR and sends it to Bob.
Bob decrypts the message.